In [15]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from datetime import datetime
import json
from string import digits
import re
from selenium import webdriver
from time import sleep

#SET CRHOME DRIVER

#work comp
#driver_path = webdriver.Chrome("C:\Users\dbell\Data Science [Github]/chromedriver.exe")

#home comp
#driver_path = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

#collecting primary articles
stories = []
def getTheGoodStuff(newsstories):
    global stories
    global i
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url,
            "sort_rank" : i}
        stories.append(d)
        for related in data.find_all(class_="related-item"):
            related_htmlatag = related.find("a")
            related_headline = related_htmlatag.getText()
            related_url = related_htmlatag.get("href")
            r = {"headline" : related_headline,
                 "url" : related_url,
                 "sort_rank" : i+1}
            stories.append(r)
            
def scrapeWebsites():
    global stories
    global i
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for i in range(0, 20):
        foundstories = soup.find_all("article", class_="article story-" + str(i))
        getTheGoodStuff(foundstories)
    
scrapeWebsites()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

fox_domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.foxnews.com/')[-1].split('/')[0]
    if domain_items[0] in ['video','foxbusiness','nation']:
        fox_domains.append(domain_items[0])
    else:
        fox_domains.append(url_item)
articles['article_type'] = fox_domains

#collecting Opinion articles
sidebar_stories = []

def opinion_getTheGoodStuff(newsstories):
    global sidebar_stories
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url}
        sidebar_stories.append(d)

def opinion_scrapeWebsites():
    global sidebar_stories
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    foundstories = soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")
    for article in foundstories:
        opinion_getTheGoodStuff(foundstories)
        
opinion_scrapeWebsites()
sidebar_articles = pd.DataFrame(sidebar_stories)
sidebar_articles.drop_duplicates(inplace=True)
sort_ranks = range(sidebar_articles.shape[0]+1)
del sort_ranks[0]
sidebar_articles["sort_rank"] = sort_ranks
sidebar_articles['date'] = datetime.today().strftime('%Y-%m-%d')
sidebar_articles['year'] = datetime.today().year
sidebar_articles['month'] = datetime.today().month
sidebar_articles['day'] = datetime.today().day
sidebar_articles['hour'] = datetime.today().hour
sidebar_articles['minute'] = datetime.today().minute

sidebar_articles['article_type'] = "opinion"

all_articles = pd.concat([articles,sidebar_articles],sort=False)

#harvesting article text
article_text = []

fox_driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in all_articles.url:
    try:
        fox_driver.get(url)
        sleep(1) #wait 1 second
        html = fox_driver.page_source
        soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
        article_text.append(soup.find_all('p'))
    except:
        article_text.append('Error')

fox_driver.close()    
all_articles['article_text'] = article_text

foxnews_articles = pd.read_csv('foxnews.csv')
print "foxnews articles current shape:"
print foxnews_articles.shape[0]
conc = pd.concat([foxnews_articles,all_articles],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('foxnews.csv', encoding = 'utf-8')
print "foxnews new shape:"
print conc.shape[0]
print ""

##--------------------------------------CNN scraper----------------------------------------------
url = "https://www.cnn.com/"
html = urllib.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

#some code to grab the relevant chunk of html containing articles
scripts = list(soup.find_all("script"))
n=0
for script in scripts:
    if n==0:
        longest_script = script
        n=1
    else:
        if len(str(script)) > len(str(longest_script)):
            longest_script = script
        
cnn_content = longest_script.string

#distilling down to the list of dictionaries containing article content
cnn_list = []
for item in cnn_content.split('{"articleList":'):
    for split in item.split(']}'):
        cnn_list.append(split)

cnn_articles = []        
for stuff in cnn_list:
    if stuff[:8] == '[{"uri":':
        cnn_articles.append(stuff.encode("utf-8"))

articles = cnn_articles[0].replace("'[","") + "]"

article_data = json.loads(articles)

data = []

for article in article_data:
    article_dict = dict(article)
    data.append([article_dict['headline'],article_dict['description'],article_dict['uri'].encode("utf-8"),article_dict['duration'],article_dict['layout'],article_dict['thumbnail']])
    
df = pd.DataFrame(data,columns=['headline','description','url','duration','layout','thumbnail'])

df['date'] = datetime.today().strftime('%Y-%m-%d')
df['year'] = datetime.today().year
df['month'] = datetime.today().month
df['day'] = datetime.today().day
df['hour'] = datetime.today().hour
df['minute'] = datetime.today().minute

sort_ranks = []

for index, record in df.iterrows():
    if record['headline'][:8] == '<strong>':
        sort_ranks.append(1)
    else:
        sort_ranks.append(sort_ranks[-1]+1)
        
df['sort_rank'] = sort_ranks

article_types = []

for index, record in df.iterrows():
    if record['duration'] <> '':
        article_types.append(record['url'].strip('/videos/').split('/')[0])
    else:
        article_types.append(record['url'].translate(None, digits).strip('////').split('/')[0])

df['article_type'] = article_types

#harvesting article text
article_text = []

cnn_driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in df.url:
    try:
        cnn_driver.get('https://www.cnn.com' + url)
        sleep(1) #wait 1 second
        html = cnn_driver.page_source
        soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
        article_text.append(soup.find_all(class_=re.compile("paragraph")))
    except:
        article_text.append('Error')

cnn_driver.close()    
df['article_text'] = article_text

cnn_articles = pd.read_csv('cnn.csv')
print "cnn articles current shape:"
print cnn_articles.shape[0]
conc = pd.concat([cnn_articles,df],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('cnn.csv', encoding = 'utf-8')
print "cnn new shape:"
print conc.shape[0]
print ""

#-----------------------------------------MSNBC Scraper----------------------------------------------------

stories = []
i = 0
def getTheGoodStuff(newsstories):
    global stories
    global i
    if newsstories.find("a") <> None:
        i=i+1
        htmlatag = newsstories.find("a")
        url = htmlatag.get("href")
        headline = htmlatag.getText()
        d = {"headline" : headline,
             "url" : url,
             "sort_rank" : i}
        stories.append(d)
            
def scrapeMSNBC():
    global stories
    global newsstories
    global i
    # Getting stories from MSNBC
    msnbc = "https://www.msnbc.com/"
    page = urllib.urlopen(msnbc)
    r  = requests.get(msnbc)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for htmlstuff in soup.find_all(class_=re.compile("title")):
        getTheGoodStuff(htmlstuff)
        
scrapeMSNBC()

#grabbing "related" type stories using very similar approach
def getrelated(newsstories):
    global stories
    if newsstories.find("a") <> None:
        htmlatag = newsstories.find("a")
        url = htmlatag.get("href")
        headline = htmlatag.getText()
        d = {"headline" : headline,
             "url" : url,
             "sort_rank" : 3}
        stories.append(d)
            
def scrapeMSNBC_related():
    global stories
    global newsstories
    # Getting stories from MSNBC
    msnbc = "https://www.msnbc.com/"
    page = urllib.urlopen(msnbc)
    r  = requests.get(msnbc)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for htmlstuff in soup.find_all(class_=re.compile("related")):
        getrelated(htmlstuff)
                                   
scrapeMSNBC_related()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.msnbc.com/msnbc/')[-1].split('https://www.msnbc.com/')[-1].split('http://www.msnbc.com/')[-1].split('https://www.nbcnews.com/politics/')[-1].split('https://www.nbcnews.com/news/')[-1].split('https://www.nbcnews.com/tech/')[-1].split('https://www.nbcnews.com/better/')[-1].split('https://www.nbcnews.com/mach/')[-1].split('https://www.nbcnews.com/think/')[-1].split('https://www.nbcnews.com/feature/')[-1].split('https://www.nbcnews.com/')[-1].split('https://podcasts.apple.com/')[-1].split('/')[0]
    domains.append(url_item)
articles['domain'] = domains

#Now that we have the article URLs, now we use chrome driver to fetch text from each article
msnbc_text = []

msnbc_driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in articles.url:
    try:
        msnbc_driver.get(url)
        sleep(1) #wait 1 second
        html = msnbc_driver.page_source
        soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
        msnbc_text.append(soup.find_all('p'))
    except:
        msnbc_text.append('Error')

msnbc_driver.close()    
articles['article_text'] = msnbc_text

msnbc_articles = pd.read_csv('msnbc.csv')
print "msnbc articles current shape:"
print msnbc_articles.shape[0]
conc = pd.concat([msnbc_articles,articles],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('msnbc.csv', encoding = 'utf-8')
print "msnbc new shape:"
print conc.shape[0]
print ""

print "Last run:"
print datetime.today()

foxnews articles current shape:
3362
foxnews new shape:
3408



TimeoutException: Message: timeout
  (Session info: chrome=75.0.3770.100)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)


In [17]:
#run CNN only
url = "https://www.cnn.com/"
html = urllib.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

#some code to grab the relevant chunk of html containing articles
scripts = list(soup.find_all("script"))
n=0
for script in scripts:
    if n==0:
        longest_script = script
        n=1
    else:
        if len(str(script)) > len(str(longest_script)):
            longest_script = script
        
cnn_content = longest_script.string

#distilling down to the list of dictionaries containing article content
cnn_list = []
for item in cnn_content.split('{"articleList":'):
    for split in item.split(']}'):
        cnn_list.append(split)

cnn_articles = []        
for stuff in cnn_list:
    if stuff[:8] == '[{"uri":':
        cnn_articles.append(stuff.encode("utf-8"))

articles = cnn_articles[0].replace("'[","") + "]"

article_data = json.loads(articles)

data = []

for article in article_data:
    article_dict = dict(article)
    data.append([article_dict['headline'],article_dict['description'],article_dict['uri'].encode("utf-8"),article_dict['duration'],article_dict['layout'],article_dict['thumbnail']])
    
df = pd.DataFrame(data,columns=['headline','description','url','duration','layout','thumbnail'])

df['date'] = datetime.today().strftime('%Y-%m-%d')
df['year'] = datetime.today().year
df['month'] = datetime.today().month
df['day'] = datetime.today().day
df['hour'] = datetime.today().hour
df['minute'] = datetime.today().minute

sort_ranks = []

for index, record in df.iterrows():
    if record['headline'][:8] == '<strong>':
        sort_ranks.append(1)
    else:
        sort_ranks.append(sort_ranks[-1]+1)
        
df['sort_rank'] = sort_ranks

article_types = []

for index, record in df.iterrows():
    if record['duration'] <> '':
        article_types.append(record['url'].strip('/videos/').split('/')[0])
    else:
        article_types.append(record['url'].translate(None, digits).strip('////').split('/')[0])

df['article_type'] = article_types

#harvesting article text
article_text = []

cnn_driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in df.url:
    try:
        cnn_driver.get('https://www.cnn.com' + url)
        sleep(1) #wait 1 second
        html = cnn_driver.page_source
        soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
        article_text.append(soup.find_all(class_=re.compile("paragraph")))
    except:
        article_text.append('Error')

cnn_driver.close()    
df['article_text'] = article_text

cnn_articles = pd.read_csv('cnn.csv')
print "cnn articles current shape:"
print cnn_articles.shape[0]
print ""
conc = pd.concat([cnn_articles,df],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('cnn2.csv', encoding = 'utf-8')
print "cnn new shape:"
print conc.shape[0]
print ""

print "Last run:"
print datetime.today()

cnn articles current shape:
2723

cnn new shape:
2777

Last run:
2019-07-03 08:35:43.907000


In [18]:
#run MSNBC only

stories = []
i = 0
def getTheGoodStuff(newsstories):
    global stories
    global i
    if newsstories.find("a") <> None:
        i=i+1
        htmlatag = newsstories.find("a")
        url = htmlatag.get("href")
        headline = htmlatag.getText()
        d = {"headline" : headline,
             "url" : url,
             "sort_rank" : i}
        stories.append(d)
            
def scrapeMSNBC():
    global stories
    global newsstories
    global i
    # Getting stories from MSNBC
    msnbc = "https://www.msnbc.com/"
    page = urllib.urlopen(msnbc)
    r  = requests.get(msnbc)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for htmlstuff in soup.find_all(class_=re.compile("title")):
        getTheGoodStuff(htmlstuff)
        
scrapeMSNBC()

#grabbing "related" type stories using very similar approach
def getrelated(newsstories):
    global stories
    if newsstories.find("a") <> None:
        htmlatag = newsstories.find("a")
        url = htmlatag.get("href")
        headline = htmlatag.getText()
        d = {"headline" : headline,
             "url" : url,
             "sort_rank" : 3}
        stories.append(d)
            
def scrapeMSNBC_related():
    global stories
    global newsstories
    # Getting stories from MSNBC
    msnbc = "https://www.msnbc.com/"
    page = urllib.urlopen(msnbc)
    r  = requests.get(msnbc)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for htmlstuff in soup.find_all(class_=re.compile("related")):
        getrelated(htmlstuff)
                                   
scrapeMSNBC_related()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.msnbc.com/msnbc/')[-1].split('https://www.msnbc.com/')[-1].split('http://www.msnbc.com/')[-1].split('https://www.nbcnews.com/politics/')[-1].split('https://www.nbcnews.com/news/')[-1].split('https://www.nbcnews.com/tech/')[-1].split('https://www.nbcnews.com/better/')[-1].split('https://www.nbcnews.com/mach/')[-1].split('https://www.nbcnews.com/think/')[-1].split('https://www.nbcnews.com/feature/')[-1].split('https://www.nbcnews.com/')[-1].split('https://podcasts.apple.com/')[-1].split('/')[0]
    domains.append(url_item)
articles['domain'] = domains

#Now that we have the article URLs, now we use chrome driver to fetch text from each article
msnbc_text = []

msnbc_driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in articles.url:
    msnbc_driver.get(url)
    sleep(1) #wait 1 second
    html = msnbc_driver.page_source
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    msnbc_text.append(soup.find_all('p'))

msnbc_driver.close()    
articles['article_text'] = msnbc_text

msnbc_articles = pd.read_csv('msnbc.csv')
print "msnbc articles current shape:"
print msnbc_articles.shape[0]
print ""
conc = pd.concat([msnbc_articles,articles],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('msnbc.csv', encoding = 'utf-8')
print "msnbc new shape:"
print conc.shape[0]
print ""

print "Last run:"
print datetime.today()

msnbc articles current shape:
399

msnbc new shape:
467

Last run:
2019-07-03 11:16:07.858000
